In [1]:
import pandas as pd
import json
import os
import requests
import csv
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:

file_path = '/Users/katigbakv/Desktop/border.csv'
data = pd.read_csv(file_path)
print(data)


           Port Name         State  Port Code            Border      Date  \
0               Roma         Texas       2310  US-Mexico Border  Dec 2023   
1            Del Rio         Texas       2302  US-Mexico Border  Dec 2023   
2       Willow Creek       Montana       3325  US-Canada Border  Jan 2024   
3           Whitlash       Montana       3321  US-Canada Border  Jan 2024   
4             Ysleta         Texas       2401  US-Mexico Border  Jan 2024   
...              ...           ...        ...               ...       ...   
395633      Portland         Maine        101  US-Canada Border  Apr 1996   
395634   Trout River      New York        715  US-Canada Border  Oct 1997   
395635       Raymond       Montana       3301  US-Canada Border  Jul 1996   
395636      Westhope  North Dakota       3419  US-Canada Border  Mar 1996   
395637      Boundary    Washington       3015  US-Canada Border  Jul 1997   

                            Measure   Value  Latitude  Longitude  \
0      

In [3]:
border_df = pd.DataFrame(data)
border_df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitude,Longitude,Point
0,Roma,Texas,2310,US-Mexico Border,Dec 2023,Buses,46,26.404,-99.019,POINT (-99.018981 26.403928)
1,Del Rio,Texas,2302,US-Mexico Border,Dec 2023,Trucks,6552,29.327,-100.928,POINT (-100.927612 29.326784)
2,Willow Creek,Montana,3325,US-Canada Border,Jan 2024,Pedestrians,2,49.000,-109.731,POINT (-109.731333 48.999972)
3,Whitlash,Montana,3321,US-Canada Border,Jan 2024,Personal Vehicles,29,48.997,-111.258,POINT (-111.257916 48.99725)
4,Ysleta,Texas,2401,US-Mexico Border,Jan 2024,Personal Vehicle Passengers,521714,31.673,-106.335,POINT (-106.335449846028 31.6731261376859)


In [4]:
border_df['Date'].nunique()

345

In [21]:
# Convert 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'], format='%b %Y', errors='coerce')

# Extract year and month
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

# Group data by year and month
monthly_data = data.groupby(['Year', 'Month']).agg({'Value': 'sum'}).reset_index()

# Pivot table for aggregated values
monthly_pivot = monthly_data.pivot_table(index=['Year', 'Month'], values='Value', fill_value=0)
monthly_pivot.reset_index(inplace=True)

# Combine Year and Month into a single column
monthly_pivot['Month-Year'] = monthly_pivot['Month'].apply(lambda x: f"{x:02d}") + '-' + monthly_pivot['Year'].astype(str)

# Drop the original Year and Month columns
monthly_pivot.drop(columns=['Year', 'Month'], inplace=True)

# Set 'Month-Year' as the index
monthly_pivot.set_index('Month-Year', inplace=True)

# Display
monthly_pivot.head(12)

,Value
Month-Year,
01-1996,32550342.0
02-1996,32212625.0
03-1996,35509073.0
04-1996,35690321.0
05-1996,37644486.0
06-1996,37731632.0
07-1996,40333161.0
08-1996,40728924.0
09-1996,36195902.0


In [14]:

data['Date'] = pd.to_datetime(data['Date'], format='%b %Y', errors='coerce')


data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month


monthly_data = data.groupby(['Year', 'Month']).agg({'Value': 'sum'}).reset_index()


monthly_pivot = monthly_data.pivot_table(index=['Year', 'Month'], values='Value', fill_value=0)
monthly_pivot.reset_index(inplace=True)
monthly_pivot.head(12)


,Year,Month,Value
0,1996,1,32550342.0
1,1996,2,32212625.0
2,1996,3,35509073.0
3,1996,4,35690321.0
4,1996,5,37644486.0
5,1996,6,37731632.0
6,1996,7,40333161.0
7,1996,8,40728924.0
8,1996,9,36195902.0
9,1996,10,35913743.0


In [16]:
import matplotlib.pyplot as plt


data['Date'] = pd.to_datetime(data['Date'], format='%b %Y', errors='coerce')


data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month


monthly_data = data.groupby(['Year', 'Month', 'Measure']).agg({'Value': 'sum'}).reset_index()


monthly_pivot = monthly_data.pivot_table(index=['Year', 'Month'], columns='Measure', values='Value', fill_value=0)
monthly_pivot.reset_index(inplace=True)
monthly_pivot.tail(12)

Measure,Year,Month,Bus Passengers,Buses,Pedestrians,Personal Vehicle Passengers,Personal Vehicles,Rail Containers Empty,Rail Containers Loaded,Train Passengers,Trains,Truck Containers Empty,Truck Containers Loaded,Trucks
333,2023,10,194020.0,14030.0,3217446.0,14700428.0,8233705.0,124235.0,181338.0,18204.0,2745.0,310065.0,1315547.0,1122369.0
334,2023,11,197805.0,12936.0,3602831.0,14286168.0,7894622.0,119855.0,177605.0,6714.0,2669.0,309369.0,1246393.0,1075004.0
335,2023,12,257473.0,16340.0,3839019.0,14986566.0,7956017.0,117061.0,185598.0,26007.0,2704.0,278472.0,1095149.0,948396.0
336,2024,1,198052.0,9554.0,2078103.0,13695051.0,7567604.0,118898.0,170548.0,40767.0,2681.0,319411.0,1482444.0,1069643.0
337,2024,2,176069.0,12655.0,3270197.0,13133500.0,7396203.0,122510.0,183506.0,6269.0,2709.0,307820.0,1501242.0,1066106.0
338,2024,3,229490.0,13688.0,3513634.0,15158027.0,8157284.0,132524.0,200115.0,7545.0,2873.0,302813.0,1562001.0,1090651.0
339,2024,4,244002.0,14693.0,3450942.0,14438307.0,8071487.0,131096.0,190206.0,7462.0,2802.0,334098.0,1640532.0,1157655.0
340,2024,5,307470.0,17345.0,3543073.0,15284605.0,8477030.0,129879.0,191267.0,36832.0,2817.0,340704.0,1635849.0,1159855.0
341,2024,6,305763.0,17669.0,3248149.0,15457524.0,8369929.0,119686.0,172350.0,42508.0,2618.0,318747.0,1528298.0,1083511.0
342,2024,7,384632.0,18694.0,3556580.0,17109259.0,8847466.0,126761.0,168875.0,48948.0,2680.0,340642.0,1543136.0,1114592.0


In [18]:
import matplotlib.pyplot as plt

# Convert 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'], format='%b %Y', errors='coerce')

# Extract year and month
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

# Group data by year, month, and measure
monthly_data = data.groupby(['Year', 'Month', 'Measure']).agg({'Value': 'sum'}).reset_index()

# Pivot table for measures
monthly_pivot = monthly_data.pivot_table(index=['Year', 'Month'], columns='Measure', values='Value', fill_value=0)
monthly_pivot.reset_index(inplace=True)

# Combine Year and Month into a single column
monthly_pivot['Month-Year'] = monthly_pivot['Month'].apply(lambda x: f"{x:02d}") + '-' + monthly_pivot['Year'].astype(str)

# Drop the original Year and Month columns
monthly_pivot.drop(columns=['Year', 'Month'], inplace=True)

# Set 'Month-Year' as the index
monthly_pivot.set_index('Month-Year', inplace=True)

# Display the last 12 rows
monthly_pivot.head(12)


Measure,Bus Passengers,Buses,Pedestrians,Personal Vehicle Passengers,Personal Vehicles,Rail Containers Empty,Rail Containers Loaded,Train Passengers,Trains,Truck Containers Empty,Truck Containers Loaded,Trucks
Month-Year,,,,,,,,,,,,
01-1996,299911.0,17873.0,3138859.0,20181055.0,7711801.0,22422.0,44353.0,13803.0,3084.0,130941.0,311889.0,674351.0
02-1996,320976.0,18566.0,2994503.0,20095676.0,7626155.0,21602.0,42141.0,12643.0,3175.0,120938.0,263897.0,692353.0
03-1996,359668.0,21551.0,3508484.0,21878001.0,8524704.0,26677.0,46848.0,13769.0,3465.0,128635.0,278244.0,719027.0
04-1996,454548.0,23346.0,3085717.0,22654384.0,8290365.0,23280.0,42915.0,16904.0,3292.0,123626.0,281492.0,690452.0
05-1996,562319.0,27565.0,2945140.0,23919187.0,8926048.0,23013.0,41777.0,17564.0,3363.0,139160.0,280927.0,758423.0
06-1996,569653.0,27960.0,2767772.0,24130611.0,9060954.0,20366.0,36003.0,23691.0,3119.0,133516.0,240207.0,717780.0
07-1996,621553.0,30714.0,2878658.0,25981113.0,9641543.0,17029.0,30680.0,30317.0,3144.0,143063.0,246960.0,708387.0
08-1996,623376.0,30823.0,2914940.0,26163054.0,9766780.0,23185.0,39017.0,32205.0,3234.0,136292.0,250314.0,745704.0
09-1996,521521.0,26367.0,2675729.0,23287346.0,8536175.0,21792.0,39409.0,19873.0,3222.0,121134.0,231768.0,711566.0
